# DEEP NEURAL NETWORKS - ASSIGNMENT 2: CNN FOR IMAGE CLASSIFICATION

## Convolutional Neural Networks: Custom Implementation vs Transfer Learning

STUDENT INFORMATION (REQUIRED - DO NOT DELETE)

BITS ID: [2025AA05444]

Name: [PRASAD SHIVAJI KULKARNI]

Email: [2025aa05444@wilp.bits-pilani.ac.in]

Date: [08/02/2026]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
import shutil
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time
import json


In [ ]:
# Deep learning frameworks (choose Keras or PyTorch)
# For image processing
from PIL import Image
import cv2

In [ ]:
# Download Cats and Dogs dataset
dataset_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=dataset_url, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

class_names = train_dataset.class_names
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [ ]:
dataset_name = "Cats vs Dogs (Filtered)"
dataset_source = "TensorFlow Datasets / Microsoft"
n_samples = 3000
n_classes = len(class_names)
samples_per_class = "1000 per class (balanced)"
image_shape = [160, 160, 3]
problem_type = "classification"


In [ ]:
primary_metric = "accuracy"
metric_justification = "Accuracy is chosen because the dataset is balanced."


In [ ]:
print("DATASET INFORMATION")
print(f"Dataset: {dataset_name}")
print(f"Source: {dataset_source}")
print(f"Total Samples: {n_samples}")
print(f"Number of Classes: {n_classes}")
print(f"Samples per Class: {samples_per_class}")
print(f"Image Shape: {image_shape}")
print(f"Primary Metric: {primary_metric}")
print(f"Metric Justification: {metric_justification}")

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
plt.show()

# Class distribution
plt.figure(figsize=(6, 4))
plt.bar(class_names, [1000, 1000])
plt.title('Class Distribution')
plt.show()


### 1.3 Data Preprocessing
- TODO: Resize images to consistent size
- TODO: Normalize pixel values
- TODO: Split into train/test (90/10 or 85/15)

In [ ]:
train_test_ratio = "67/33 (approx)"
train_samples = 2000
test_samples = 200


In [ ]:
print(f"\nTrain/Test Split: {train_test_ratio}")
print(f"Training Samples: {train_samples}")
print(f"Test Samples: {test_samples}")

### 2.1 Custom CNN Architecture Design
- TODO: Define your CNN architecture
- TODO: Ensure Global Average Pooling is included (MANDATORY)
- TODO: Use Conv2D, MaxPooling2D/AvgPooling2D, GlobalAveragePooling2D, Dense

In [ ]:
def build_custom_cnn(input_shape, n_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    # Global Average Pooling (MANDATORY)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid') if n_classes == 2 else layers.Dense(n_classes, activation='softmax')

    model = models.Model(inputs, outputs)
    return model


In [ ]:
# TODO: Create model instance
custom_cnn = build_custom_cnn(image_shape, n_classes)

In [ ]:
custom_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy() if n_classes==2 else 'categorical_crossentropy',
              metrics=['accuracy'])


### 2.2 Train Custom CNN

In [ ]:
print("\nCUSTOM CNN TRAINING")
# Track training time
custom_cnn_start_time = time.time()

In [ ]:
history_custom = custom_cnn.fit(train_dataset,
                                validation_data=validation_dataset,
                                epochs=15)


In [ ]:
custom_cnn_training_time = time.time() - custom_cnn_start_time

In [ ]:
custom_cnn_initial_loss = history_custom.history['loss'][0]
custom_cnn_final_loss = history_custom.history['loss'][-1]


In [ ]:
print(f"Training completed in {custom_cnn_training_time:.2f} seconds")
print(f"Initial Loss: {custom_cnn_initial_loss:.4f}")
print(f"Final Loss: {custom_cnn_final_loss:.4f}")

In [ ]:
print("\nCUSTOM CNN EVALUATION")

### 2.3 Evaluate Custom CNN
- TODO: Make predictions on test set
- TODO: Calculate all 4 required metrics

In [ ]:
y_true = []
y_pred = []
for images, labels in test_dataset:
    preds = custom_cnn.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten() if n_classes==2 else np.argmax(preds, axis=1))

custom_cnn_accuracy = accuracy_score(y_true, y_pred)
custom_cnn_precision = precision_score(y_true, y_pred, average='binary' if n_classes==2 else 'macro')
custom_cnn_recall = recall_score(y_true, y_pred, average='binary' if n_classes==2 else 'macro')
custom_cnn_f1 = f1_score(y_true, y_pred, average='binary' if n_classes==2 else 'macro')


In [ ]:
print("\nCustom CNN Performance:")
print(f"Accuracy:  {custom_cnn_accuracy:.4f}")
print(f"Precision: {custom_cnn_precision:.4f}")
print(f"Recall:    {custom_cnn_recall:.4f}")
print(f"F1-Score:  {custom_cnn_f1:.4f}")

In [ ]:
acc = history_custom.history['accuracy']
val_acc = history_custom.history['val_accuracy']
loss = history_custom.history['loss']
val_loss = history_custom.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.show()


### 3.1 Load Pre-trained Model and Modify Architecture

In [ ]:
print("\n" + "="*70)
print("TRANSFER LEARNING IMPLEMENTATION")

In [ ]:
pretrained_model_name = "ResNet50"


In [ ]:
def build_transfer_learning_model(base_model_name, input_shape, n_classes):
    base_model = tf.keras.applications.ResNet50(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x) if n_classes == 2 else layers.Dense(n_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy' if n_classes==2 else 'categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# TODO: Create transfer learning model
transfer_model = build_transfer_learning_model(pretrained_model_name, image_shape, n_classes)

In [ ]:
frozen_layers = len(transfer_model.layers[1].layers) # Base model layers
trainable_layers = len(transfer_model.layers) - 1 # Top layers
total_parameters = transfer_model.count_params()
trainable_parameters = sum([w.shape.num_elements() for w in transfer_model.trainable_weights])


In [ ]:
print(f"Base Model: {pretrained_model_name}")
print(f"Frozen Layers: {frozen_layers}")
print(f"Trainable Layers: {trainable_layers}")
print(f"Total Parameters: {total_parameters:,}")
print(f"Trainable Parameters: {trainable_parameters:,}")
print(f"Using Global Average Pooling: YES")

### 3.2 Train Transfer Learning Model

In [ ]:
print("\nTraining Transfer Learning Model...")

In [ ]:
# Training configuration
tl_learning_rate = 0.001
tl_epochs = 10
tl_batch_size = 32
tl_optimizer = "Adam"

In [ ]:
# Track training time
tl_start_time = time.time()

In [ ]:
history_tl = transfer_model.fit(train_dataset,
                                validation_data=validation_dataset,
                                epochs=10)


In [ ]:
tl_training_time = time.time() - tl_start_time

In [ ]:
tl_initial_loss = history_tl.history['loss'][0]
tl_final_loss = history_tl.history['loss'][-1]


In [ ]:
print(f"Training completed in {tl_training_time:.2f} seconds")
print(f"Initial Loss: {tl_initial_loss:.4f}")
print(f"Final Loss: {tl_final_loss:.4f}")

### 3.3 Evaluate Transfer Learning Model
- TODO: Make predictions on test set
- TODO: Calculate all 4 required metrics

In [ ]:
y_true_tl = []
y_pred_tl = []
for images, labels in test_dataset:
    preds = transfer_model.predict(images, verbose=0)
    y_true_tl.extend(labels.numpy())
    y_pred_tl.extend((preds > 0.5).astype(int).flatten() if n_classes==2 else np.argmax(preds, axis=1))

tl_accuracy = accuracy_score(y_true_tl, y_pred_tl)
tl_precision = precision_score(y_true_tl, y_pred_tl, average='binary' if n_classes==2 else 'macro')
tl_recall = recall_score(y_true_tl, y_pred_tl, average='binary' if n_classes==2 else 'macro')
tl_f1 = f1_score(y_true_tl, y_pred_tl, average='binary' if n_classes==2 else 'macro')


In [ ]:
print("\nTransfer Learning Performance:")
print(f"Accuracy:  {tl_accuracy:.4f}")
print(f"Precision: {tl_precision:.4f}")
print(f"Recall:    {tl_recall:.4f}")
print(f"F1-Score:  {tl_f1:.4f}")

In [ ]:
acc = history_tl.history['accuracy']
val_acc = history_tl.history['val_accuracy']
loss = history_tl.history['loss']
val_loss = history_tl.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('TL Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('TL Training and Validation Loss')
plt.show()

cm = confusion_matrix(y_true_tl, y_pred_tl)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('TL Confusion Matrix')
plt.show()


### 4.1 Metrics Comparison

In [ ]:
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Training Time (s)', 'Parameters'],
    'Custom CNN': [
        custom_cnn_accuracy,
        custom_cnn_precision,
        custom_cnn_recall,
        custom_cnn_f1,
        custom_cnn_training_time,
        custom_cnn.count_params()
    ],
    'Transfer Learning': [
        tl_accuracy,
        tl_precision,
        tl_recall,
        tl_f1,
        tl_training_time,
        trainable_parameters
    ]
})


In [ ]:
print(comparison_df.to_string(index=False))

In [ ]:
comparison_df.set_index('Metric')[['Custom CNN', 'Transfer Learning']].iloc[:4].plot(kind='bar')
plt.title('Performance Comparison')
plt.show()


In [ ]:
analysis_text = """
1. Performance: Transfer Learning (ResNet50) achieved higher accuracy and f1_score compared to the Custom CNN. Pre-trained weights provided a strong feature extractor.
2. Pre-training Impact: Using pre-trained weights significantly faster convergence. The model started with high accuracy, whereas Custom CNN needed more epochs.
3. GAP Effect: Global Average Pooling reduced parameters significantly compared to Flatten+Dense, preventing overfitting and reducing computational cost.
4. Cost: Custom CNN is lightweight in parameters but takes longer to converge. TL has more parameters (in base) but trainable parameters are few, making fine-tuning fast.
5. Insights: Transfer learning is superior for small datasets like this, leveraging learned features from ImageNet. Custom CNN requires more data/epochs to match performance.
"""


In [ ]:
# REQUIRED: Print analysis with word count
print("ANALYSIS")
print(analysis_text)
print(f"Analysis word count: {len(analysis_text.split())} words")
if len(analysis_text.split()) > 200:
    print("  Warning: Analysis exceeds 200 words (guideline)")
else:
    print(" Analysis within word count guideline")

In [ ]:
        'custom_cnn': {
            'framework': framework_used,
            'architecture': {
                'conv_layers': 3,
                'pooling_layers': 3,
                'has_global_average_pooling': True,
                'output_layer': 'sigmoid',
                'total_parameters': custom_cnn.count_params()
            },
            'training_config': {
                'learning_rate': 0.001,
                'n_epochs': 15,
                'batch_size': 32,
                'optimizer': 'Adam',
                'loss_function': 'binary_crossentropy'
            },
            'initial_loss': custom_cnn_initial_loss,
            'final_loss': custom_cnn_final_loss,
            'training_time_seconds': custom_cnn_training_time,
            'accuracy': custom_cnn_accuracy,
            'precision': custom_cnn_precision,
            'recall': custom_cnn_recall,
            'f1_score': custom_cnn_f1
        },


In [ ]:
# Generate and print results
try:
    assignment_results = get_assignment_results()
    print("ASSIGNMENT RESULTS SUMMARY")
    print(json.dumps(assignment_results, indent=2))

except Exception as e:
    print(f"\n  ERROR generating results: {str(e)}")
    print("Please ensure all variables are properly defined")

In [ ]:
# Display system information
import platform
import sys
from datetime import datetime

In [ ]:
print("ENVIRONMENT INFORMATION")
print("\n  REQUIRED: Add screenshot of your Google Colab/BITS Virtual Lab")
print("showing your account details in the cell below this one.")

# include the screen shot here